In [ ]:
import autograd.numpy as np
from autograd import grad
from autograd.numpy import linalg, dot
from autograd.numpy.random import multivariate_normal, normal
%matplotlib inline
from matplotlib import pyplot as plt

from ChainNode import ChainNode
from Neighbour import Neighbour 

In [ ]:
#initialise parameters
N = 20 # Number of iterations
M = 2 # dimensionality of state
L = 2 # dimensionality of output
e = 1e-5 # convergence condition

#Kalman Parameters, linear motion estimation
t_s = 0.1 # sample period
s_p = 1 # process noise variance
s_m = 0.1 # measurement noise variance

#model parameters
A = np.asarray([[1,t_s],[0,1]]) # state transition
B = np.eye(M)
C = np.eye(M)
Q = np.asarray([[0.5,0.1],[0.1,0.5]])
R = np.asarray([[0.5,0.1],[0.1,0.5]])

#state variables
u = multivariate_normal(np.zeros([2]),Q).reshape([2,1])
z = np.asarray([[1],[1]]) # initial statev
v = multivariate_normal(np.zeros([2]),R).reshape([2,1])
y = np.dot(C,z) + v #initial output

#Kalman filter variables 
P_init = np.asarray([[0.5,0.1],[0.1,0.5]])

P_post = np.copy(P_init)
z_post = normal(0,1,[2,1])
z_post = z

#PDMM Parameters
Z = z # state history
U = u # noise history
Y = y # output history

Z_post = z_post #posterior estimate history

# E = linalg.inv(np.block([[Q,np.zeros(M,M)],[np.zeros([M,M]),R]]))
p = 0.001 #step size

In [ ]:
# Run kalman simulation
for i in range(N):
    
    z_prior = dot(A,z_post) 
    P_prior = dot(A,dot(P_post,A.T)) + Q
    
    u = multivariate_normal(np.zeros([2]),Q).reshape([2,1])
    v = multivariate_normal(np.zeros([2]),R).reshape([2,1])
    z = np.dot(A,z) + u
    y = np.dot(C,z) + v
    
    #Perform Kalman filtering, General equation needed for numerical stability
    K_int = linalg.inv(dot(C,dot(P_prior,C.T) + R))
    K = dot(P_prior,dot(C.T,K_int))
    z_post = z_prior + dot(K,(y - dot(C,z_prior)))
    P_int = np.eye(2) - dot(L,C)
    P_post = dot(dot(P_int,P_prior),P_int.T) + dot(K,dot(R,K.T))
    
    #add new information to history
    Z = np.append(Z,z,axis=1)
    Z_post = np.append(Z_post,z_post,axis=1)
    U = np.append(U,u,axis=1)
    Y = np.append(Y,y,axis=1)

In [ ]:
# Initialise PDMM Graph
Q_inv = linalg.inv(Q)
R_int = dot(C.T,dot(linalg.inv(R),C))
P_inv = linalg.inv(P_init)
a_int = dot(linalg.inv(R),C)

N_dim = 4
msg_dim = 2
p = 5e-3
d = 1e-15
N_max = 1e4

E = np.block([[Q_inv,np.zeros([2,2])],[np.zeros([2,2]),R_int]])
E_0 = np.block([[Q_inv,np.zeros([2,2])],[np.zeros([2,2]),R_int + P_inv]]) 
a_0 = np.block([np.zeros([1,2]),dot(Y[:,0],a_int)]).T

G = [ChainNode(0,N_dim,E,a_0,p,d,N_max)] 

#initialise nodes
for i in np.arange(N-1)+1:
    a_i = np.block([np.zeros([1,2]),dot(Y[:,i],a_int)]).T
    G.append(ChainNode(i,N_dim,E,a_i,p,d,N_max))

c_ij = np.zeros([2,1]) 
    
#insert forward neighbours
A_forward = np.block([-B,-A])
A_backward = np.block([np.zeros([2,2]),np.eye(2)])

P_ij = dot(A_forward,dot(linalg.inv(E_0),A_forward.T))
forward_neighbour = Neighbour(G[1],1,A_forward,c_ij,P_ij,msg_dim)
G[0].Neighbours.append(forward_neighbour)

for i in np.arange(1,N-1):
    
    backward_neighbour = Neighbour(G[i-1],i-1,A_backward,c_ij,P_ij,msg_dim)
    G[i].Neighbours.append(backward_neighbour)
    
    P_ij_int = linalg.inv(G[i].E + dot(A_backward.T,dot(linalg.inv(P_ij),A_backward)))
    P_ij = dot(A_forward,dot(P_ij_int,A_forward.T))
    forward_neighbour = Neighbour(G[i+1],i+1,A_forward,c_ij,P_ij,msg_dim)
    G[i].Neighbours.append(forward_neighbour)

P_ij_int = linalg.inv(G[i].E + dot(A_backward.T,dot(linalg.inv(P_ij),A_backward)))
P_ij = dot(A_forward,dot(P_ij_int,A_forward.T))
backward_neighbour = Neighbour(G[N-2],N-2,A_backward,c_ij,P_ij,msg_dim)
G[N-1].Neighbours.append(backward_neighbour)

In [ ]:
# Run Synchronous PDMM
X = []
x = []
for node in G:
    x.append(node.x)
X.append(x)

N_iter = 21
for i in range(N_iter):
    print(i)
    for node in G:
        node.update()

    for node in G:
        node.finalise()
        
    x = []
    for node in G:
        x.append(node.x)
    X.append(x)

In [ ]:
#collect messages
Z_pdmm = []
for i in range(N):
    Z_pdmm.append(G[i].Neighbours[0].m_ij)
Z_pdmm = np.asarray(Z_pdmm).reshape(N,2)

In [ ]:
X = np.asarray(X).reshape([N_iter+1,N,N_dim])

In [ ]:
plt.figure(figsize=(15,10))
l1, = plt.plot(Z[0,:],'x-',label='True State')
l2, = plt.plot(Z_post[0,:].T,'x-',label='Kalman Filter')
l3, = plt.plot(X[N_iter,:,2],'x-',label='Synchronous PDMM')
plt.xlabel('Iteration',fontsize=20)
plt.ylabel('Position',fontsize=20)
plt.title('Kalman Filter and PDMM Comparison',fontsize=20)
plt.legend(handles=[l1,l2,l3],fontsize=20)

In [ ]:
for j in range(N):
    plt.figure(figsize=(7,5))
    for i in range(N_dim):
        plt.plot(X[:,j,i],'x-')
        plt.plot((N_iter+1)*[Z_post[0,j]])
        plt.plot((N_iter+1)*[Z_post[1,j]])